In [ ]:
pip install "ag2[gemini]"

In [2]:
from autogen import ConversableAgent

In [4]:
from google.colab import userdata
import os
api_key=userdata.get('Gemini_main')
os.environ["GEMINI_API_KEY"] = api_key

In [5]:
llm_config = {
  "config_list": [
    {
      "api_type": "gemini",
      "model": "gemini-2.0-flash-lite",
      "api_key": api_key,
      "api_type": "google"
    }
  ],
}

In [7]:
personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent who works for TBH.AI,
    you are here to help new customers get started with our AI consultation service.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE'
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [8]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent who works for TBH.AI,
    you are here to help new customers by consulting the user doubts about AI and AI project requirements
    Your job is to gather customer's AI and AI project management requirements.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [9]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful and knowledgeable customer engagement agent at TBH.AI.
Your role is to assist users with AI and AI project management questions, providing insightful recommendations, best practices, and guidance.
Ensure that the interaction remains engaging and informative, tailoring responses to the user's specific needs.
Do not ask for unrelated information.
Return 'TERMINATE' when you have provided sufficient assistance.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [10]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [18]:
chats = [
    {
        "sender": personal_information_agent,
        "recipient": customer_proxy_agent,
        "message":
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location and what organisation the user work for?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': '','Organisation': ''}",  ## Custom summary_prompt
        },
        "max_turns": 1,
        "clear_history" : True
    },

  {
    "sender": customer_engagement_agent,
    "recipient": customer_proxy_agent,
    "message":
        "Great! How can I assist you with your AI and AI project management questions?",
    "summary_method": "reflection_with_llm",
    "max_turns": 1,
    "clear_history": False
  },

    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

In [19]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location and what organisation the user work for?

--------------------------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/autogen/agentchat/chat.py:57: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Replying as customer_proxy_agent. Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: Hello i am saish from India and work for ABC.inc
customer_proxy_agent (to Onboarding Personal Information Agent):

Hello i am saish from India and work for ABC.inc

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Customer Engagement Agent (to customer_proxy_agent):

Great! How can I assist you with your AI and AI project management questions?
Context: 
```json
{
  "name": "saish",
  "location": "India",
  "Organisation": "ABC.inc"
}
```


--------------------------------------------------------------------------------
Replying as customer_proxy_agent. Provide feedback to Customer Engagement Agent. P

In [20]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{'content': '```json\n{\n  "name": "saish",\n  "location": "India",\n  "Organisation": "ABC.inc"\n}\n```\n', 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': None}


{'content': 'The user is seeking guidance on whether to use RAG or fine-tuning for building a custom chatbot using PDF data.\n', 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': None}


{'content': 'For building a custom chatbot with PDF data, RAG is generally recommended over fine-tuning due to its flexibility, cost-effectiveness, and ease of updating with new information, particularly for someone like Saish at ABC.inc. Start with RAG and iterate.\n', 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': None}




In [21]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 8.1075e-05, 'gemini-2.0-flash-lite': {'cost': 8.1075e-05, 'prompt_tokens': 737, 'completion_tokens': 86, 'total_tokens': 823}}, 'usage_excluding_cached_inference': {'total_cost': 8.1075e-05, 'gemini-2.0-flash-lite': {'cost': 8.1075e-05, 'prompt_tokens': 737, 'completion_tokens': 86, 'total_tokens': 823}}}


{'usage_including_cached_inference': {'total_cost': 1.47e-05, 'gemini-2.0-flash-lite': {'cost': 1.47e-05, 'prompt_tokens': 100, 'completion_tokens': 24, 'total_tokens': 124}}, 'usage_excluding_cached_inference': {'total_cost': 1.47e-05, 'gemini-2.0-flash-lite': {'cost': 1.47e-05, 'prompt_tokens': 100, 'completion_tokens': 24, 'total_tokens': 124}}}


{'usage_including_cached_inference': {'total_cost': 0.00037634999999999996, 'gemini-2.0-flash-lite': {'cost': 0.00037634999999999996, 'prompt_tokens': 1218, 'completion_tokens': 950, 'total_tokens': 2168}}, 'usage_excluding_cached_inference': {'total_cost': 0.00037634999999999996, 'gem